# 安装必要包

In [1]:
pip install langchain

In [2]:
pip install openai==0.28

In [3]:
pip install webdriver_manager

In [4]:
pip install selenium

# 导入所有需要的内容

In [5]:
import openai
import os
from langchain.chat_models import ChatOpenAI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

# for test

In [48]:
def get_completion(prompt, model = "gpt-4"):
    messages = [{"role": "user", "content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message["content"]

In [49]:
get_completion("what is 1+1?")

'2'

# 用户需要输入的


In [27]:
## chatgpt4 key
os.environ["OPENAI_API_KEY"]=""
openai.api_key=""

## 威科先行的用户名和密码
# wkxx_username = ""
# wkxx_password = ""
wkxx_username = ""
wkxx_password = ""
question = "请问法律法规对于著作权侵权的赔偿标准的规定是什么？"

# langchain处理用户的输入

In [51]:
chat = ChatOpenAI(model_name='gpt-4', temperature = 0)
chat.model_name

'gpt-4'

In [52]:
prefix = """
The main task is to extract the main keywords in the question and concatenate them using the appropriate Boolean operators,
which I am asking you to do because I will be searching the database afterwards, and another important requirement is that the returned string should not be longer than 45.
"""

suffix = """
User: {query}
AI: """

examples = [
    {
        "query":"请问法律法规对于著作权侵权的赔偿标准的规定是什么？",
        "answer":"著作权 AND 侵权 AND 赔偿标准"
    },{
        "query":"实操中上海著作权相关的案例中赔偿标准是如何界定的？",
        "answer":"上海 AND 著作权 AND 案例 AND 赔偿标准"
    }
]

example_template = """
User:{query}
AI:{answer}
"""

example_prompt = PromptTemplate(
    input_variables = ["query", "answer"],
    template = example_template
)

In [53]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n\n"
)

In [54]:
# query = "请问北京的法律法规对著作权侵权的赔偿标准是什么"
whole = few_shot_prompt_template.format(query = question)
print(whole)


The main task is to extract the main keywords in the question and concatenate them using the appropriate Boolean operators, 
which I am asking you to do because I will be searching the database afterwards, and another important requirement is that the returned string should not be longer than 45.



User:请问法律法规对于著作权侵权的赔偿标准的规定是什么？
AI:著作权 AND 侵权 AND 赔偿标准



User:实操中上海著作权相关的案例中赔偿标准是如何界定的？
AI:上海 AND 著作权 AND 案例 AND 赔偿标准



User: 请问法律法规对于著作权侵权的赔偿标准的规定是什么？
AI: 


In [55]:
## 处理后的结果
asked_question = get_completion(whole)

In [56]:
print(asked_question)

法律法规 AND 著作权 AND 侵权 AND 赔偿标准


# 爬虫

In [8]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (151 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

In [34]:
import pickle

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 打开登录页面
driver.get("https://www.wkinfo.com.cn/login/index")

# 设置显式等待
wait = WebDriverWait(driver, 110)

In [36]:
# 等待用户名输入框可交互并输入用户名
username_box = wait.until(EC.element_to_be_clickable((By.ID, "firstname")))
username_box.send_keys(wkxx_username)

# 同样等待密码输入框并输入密码
password_box = wait.until(EC.element_to_be_clickable((By.ID, "lastname")))
password_box.send_keys(wkxx_password)

# 点击登录按钮
login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-sign")))
login_button.click()

In [37]:
# 保存 cookies 到文件
pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))

In [38]:
wait.until(EC.presence_of_element_located((By.ID, "wkb-search-tools")))

<selenium.webdriver.remote.webelement.WebElement (session="c5b2c64e153afdf514f54808dc060e10", element="69A261F89FE90878B063CBB5F03BBFAC_element_38")>

In [39]:
# 加载并设置 cookies
for cookie in pickle.load(open("cookies.pkl", "rb")):
    driver.add_cookie(cookie)

driver.get("https://law.wkinfo.com.cn/")
# 设置显式等待
wait = WebDriverWait(driver, 100)

In [40]:
try:
    # 尝试定位元素
    search_button = driver.find_element(By.CSS_SELECTOR, ".cg-button-field-1-8-0")
    print("元素存在")
except NoSuchElementException:
    # 如果元素不存在，捕获 NoSuchElementException 异常
    print("元素不存在")

元素存在


In [41]:
asked_question = '侵权'

In [42]:
# 定位搜索框并输入搜索内容
search_box = wait.until(EC.element_to_be_clickable((By.ID, "keyword")))
search_box.send_keys(asked_question)
# for char in "著作权 AND 案例 AND 赔偿标准":
#     search_box.send_keys(char)
#     time.sleep(0.2)  # 等待0.1秒

# # 定位搜索按钮并点击
wait = WebDriverWait(driver, 10)
# search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".wk-banner-action-bar-item.wk-button.wkb-btn-green")))
# search_button.click()
search_button = driver.find_element(By.CSS_SELECTOR, ".wk-banner-action-bar-item.wk-button.wkb-btn-green")
driver.execute_script("arguments[0].scrollIntoView(true); arguments[0].click();", search_button)


In [43]:
# 设置显式等待
wait = WebDriverWait(driver, 10)

In [44]:
# driver.current_url

In [45]:
# 设置最长等待时间为4秒
wait = WebDriverWait(driver, 8)

try:
    # 尝试在4秒内定位元素
    search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".titleHL")))
    print("元素存在")
except TimeoutException:
    # 如果在4秒内没有找到元素，捕获 TimeoutException 异常
    print("元素不存在")

元素存在


In [46]:
# 等待搜索结果的容器加载
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".wkb-list-docs-list")))

# 获取包含特定类 "titleHL" 的所有链接
links_with_titleHL = driver.find_elements(By.CSS_SELECTOR, "a .titleHL")

# 提取这些链接的 href 属性
urls = [link.find_element(By.XPATH, "..").get_attribute('href') for link in links_with_titleHL][:3]

In [47]:
# 输出前三个链接
for url in urls:
    print(url)

print(urls)

https://law.wkinfo.com.cn/legislation/detail/MTAwMDAwNzI3MjQ%3D?searchId=214825b80e854a46a160329d57bb25d4&index=1&q=%E4%BE%B5%E6%9D%83&module=&childModule=all
https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzMyMTI%3D?searchId=214825b80e854a46a160329d57bb25d4&index=2&q=%E4%BE%B5%E6%9D%83&module=&childModule=all
https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzA0NTc%3D?searchId=214825b80e854a46a160329d57bb25d4&index=3&q=%E4%BE%B5%E6%9D%83&module=&childModule=all
['https://law.wkinfo.com.cn/legislation/detail/MTAwMDAwNzI3MjQ%3D?searchId=214825b80e854a46a160329d57bb25d4&index=1&q=%E4%BE%B5%E6%9D%83&module=&childModule=all', 'https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzMyMTI%3D?searchId=214825b80e854a46a160329d57bb25d4&index=2&q=%E4%BE%B5%E6%9D%83&module=&childModule=all', 'https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzA0NTc%3D?searchId=214825b80e854a46a160329d57bb25d4&index=3&q=%E4%BE%B5%E6%9D%83&module=&childModule=all']


# 保存结果

In [71]:
pip install html2text

In [70]:
pip install playwright

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.9/612.9 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.0.1
    Uninstalling greenlet-3.0.1:
      Successfully uninstalled greenlet-3.0.1


In [78]:
urls=['https://law.wkinfo.com.cn/legislation/detail/MTAwMDAwNzI3MjQ%3D?searchId=214825b80e854a46a160329d57bb25d4&index=1&q=%E4%BE%B5%E6%9D%83&module=&childModule=all', 'https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzMyMTI%3D?searchId=214825b80e854a46a160329d57bb25d4&index=2&q=%E4%BE%B5%E6%9D%83&module=&childModule=all', 'https://law.wkinfo.com.cn/legislation/detail/MTAxMDAxMzA0NTc%3D?searchId=214825b80e854a46a160329d57bb25d4&index=3&q=%E4%BE%B5%E6%9D%83&module=&childModule=all']

In [ ]:
from langchain.document_loaders import AsyncHtmlLoader

loader = AsyncHtmlLoader(urls[0])
docs = loader.load()
# docs

In [76]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain
import os


import warnings
warnings.filterwarnings("ignore")
model_name = "gpt-4"
task_url = urls[0]
llm = OpenAI(model_name=model_name, temperature=0)

template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。

>>> {requests_result} <<<

根据网页内容，回答问题:{query}。"""

prompt = PromptTemplate(
    input_variables=["requests_result", "query"],
    template=template
)

chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt))
inputs = {
  "url": task_url,
  "query":"中华人民共和国侵权责任法第一章第一条是什么，用中文回答",
}

response = chain(inputs)
print(response['output'])


网页内容中并未提供中华人民共和国侵权责任法第一章第一条的相关信息。


In [81]:
driver.get(urls[0])

# 获取并打印网页源代码
content = driver.page_source
# print(content)

In [87]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool

# backup

In [ ]:
# # 检查是否有搜索结果
# search_results = driver.find_elements(By.CSS_SELECTOR, ".wkb-list-docs-list .ng-star-inserted a[href]")

# if search_results:
#     print("找到搜索结果，提取前三个链接。")
#     # 提取前三个搜索结果的链接
#     top_three_links = [result.get_attribute('href') for result in search_results[:3]]

#     # 遍历并访问这些链接
#     for link in top_three_links:
#         # driver.get(link)
#         print(link)
#         # ... 在此处执行您想要的操作 ...
# else:
#     print("没有找到搜索结果。")